In [1]:
import numpy as np

In [2]:
def calculate_profit(decision,P_init,n0,P_semi_product,P_product):
    #decision=[x1,x2,x3,y,z,gamma,eta]
    #半成品1和半成品2的工续完全一样，考虑为一种情况，减少决策变量
    #储存决策变量(xi:是否检测第i件零件;yi:是否检测第i件半成品;zi:是否拆解第i件不合格的半成品;gamma:是否检测成品;eta:是否拆解不合格成品)
    C=[1,1,2]#储存各零件的检测成本
    C_assemble_semi_product=8
    C_test_semi_product=4
    C_disassemble_semi_product=6
    C_assemble_product=8
    C_test_product=6
    C_disassemble_product=10
    C_market=200
    C_lose=40
    #阶段1:检测零件
    P_upgrade=[(1-decision[i])*P_init[i] for i in range(len(P_init))]
    C_test_part_total=sum([decision[i]*n0*C[i] for i in range(3)])
    #阶段2:装配半成品
    n_semi_product1=min((1-decision[0]*P_init[0])*n0,(1-decision[1]*P_init[1])*n0,(1-decision[2]*P_init[2])*n0)
    C_assemble_semi_product_total=C_assemble_semi_product*n_semi_product1
    #阶段3:是否检测半成品
    C_test_semi_product_total=decision[3]*n_semi_product1*C_test_semi_product
    #未检测前半成品的总次品率
    P_semi_product1_total=(1-P_upgrade[0])*(1-P_upgrade[1])*(1-P_upgrade[2])*P_semi_product+P_upgrade[0]+P_upgrade[1]+P_upgrade[2]-P_upgrade[0]*P_upgrade[1]-P_upgrade[1]*P_upgrade[2]-P_upgrade[0]*P_upgrade[2]+P_upgrade[0]*P_upgrade[1]*P_upgrade[2]
    #更新半成品总次品率，以便计算成品的总次品率
    P_semi_product1_total_upgrade=(1-decision[3])*P_semi_product1_total
    #阶段4:是否拆解不合格半成品
    C_disassemble_semi_product_total=(decision[4]*n_semi_product1*P_semi_product1_total)*C_disassemble_semi_product
    #阶段5:装配成品
    #可装配的成品数量
    n=np.floor(1-decision[3]*P_semi_product1_total)*n_semi_product1
    C_assemble_product_total=n*C_assemble_product
    #阶段6:是否检测成品
    P_product_total=(1-P_semi_product1_total_upgrade)*(1-P_semi_product1_total_upgrade)*(1-P_semi_product1_total_upgrade)*P_product+P_semi_product1_total_upgrade+P_semi_product1_total_upgrade+P_semi_product1_total_upgrade-P_semi_product1_total_upgrade*P_semi_product1_total_upgrade-P_semi_product1_total_upgrade*P_semi_product1_total_upgrade-P_semi_product1_total_upgrade*P_semi_product1_total_upgrade+P_semi_product1_total_upgrade*P_semi_product1_total_upgrade*P_semi_product1_total_upgrade
    C_test_product_total=decision[5]*n*C_test_product
    #阶段7:销售
    Sale=n*(1-P_product_total)*C_market
    C_lose_total=(1-decision[5])*C_lose*n*P_product_total
    #阶段8:是否拆解不合格成品
    C_disassemble_product_total=decision[6]*P_product_total*n*C_disassemble_product
    #最终的目标函数
    profit=Sale-C_lose_total-(C_test_part_total-C_assemble_semi_product_total-C_disassemble_semi_product_total-C_assemble_product_total-C_test_product_total-C_disassemble_product_total)*3+C[0]*n0*decision[0]
    n_upgrade=np.floor(decision[4]*n_semi_product1*P_semi_product1_total)
    return profit,P_upgrade,n_upgrade

In [3]:
decision=[]
for x1 in [0,1]:
    for x2 in [0,1]:
        for x3 in [0,1]:
            for y in [0,1]:
                for z in [0,1]:
                    for gamma in [0,1]:
                        for eta in [0,1]:
                            decision.append([x1,x2,x3,y,z,gamma,eta])

In [5]:
P_part_sample=0.1
P_semi_product_sample=0.1
P_product_sample=0.1
same=0
different=0
for P_part in np.linspace(0.95*P_part_sample,1.05*P_part_sample,5):
    for P_semi_product in np.linspace(0.95*P_semi_product_sample,1.05*P_semi_product_sample,5):
        for P_product in np.linspace(0.95*P_product_sample,1.05*P_product_sample,5):
            result=[]
            for i in decision:
                P_init=[P_part for i in range(3)]
                n0=10000
                profit,P_upgrade,n_upgrade=calculate_profit(i,P_init,n0,P_semi_product,P_product)
                result.append([i,profit,P_upgrade,n_upgrade])
            non_zero_index=[]
            for i in range(2**7):
                if (result[i][3]!=0)&(result[i][1]>0):
                    non_zero_index.append(i)

            result1=np.empty((2**7,2**7), dtype=object)
            for i in non_zero_index:
                for j in range(2**7):
                    profit,P_upgrade,n_upgrade=calculate_profit(decision[j],result[i][2],result[i][3],P_semi_product,P_product)
                    result1[i,j]=[decision[j],profit,P_upgrade,n_upgrade]
            non_zero_index1=[]
            for i in non_zero_index:
                for j in range(2**7):
                    if (result1[i][j][3]!=0)&(result1[i][j][1]>0):
                        non_zero_index1.append([i,j])

            result2=np.empty((2**7,2**7,2**7), dtype=object)
            for i in non_zero_index1:
                for j in range(2**7):
                    profit,P_upgrade,n_upgrade=calculate_profit(decision[j],result1[i[0]][i[1]][2],result1[i[0]][i[1]][3],P_semi_product,P_product)
                    result2[i[0],i[1],j]=[decision[j],profit,P_upgrade,n_upgrade]

            result2_value=np.zeros((2**7,2**7,2**7))
            for i in non_zero_index1:
                for j in range(2**7):
                    result2_value[i[0]][i[0]][j]=result2[i[0]][i[0]][j][1]
            result2_value_max=np.max(result2_value,axis=2)

            result1_value=np.zeros((2**7,2**7))
            for i in non_zero_index:
                for j in range(2**7):
                    result1_value[i,j]=result1[i,j][1]
                    result1_value[i,j]=result1[i,j][1]+result2_value_max[i,j]
            result1_value_max=np.max(result1_value,axis=1)
            
            result_value=np.zeros(2**7)
            for i in range(2**7):
                result_value[i]=result[i][1]
                result_value[i]=result_value[i]+result1_value_max[i]
            max_index = np.argmax(result_value)
            max_value = result_value[max_index]
            print("最大利润为:")
            print(max_value)
            print("第一阶段的决策为:")
            print(result[max_index])
            max_index1= np.argmax(result1_value[max_index])
            print("第二阶段的决策为:")
            print(result1[max_index][max_index1])
            max_index2= np.argmax(result2_value[max_index][max_index1])
            print("第三阶段的决策为:")
            print(result2[max_index][max_index1][max_index2])
            if (result[max_index][0]==[0, 0, 0, 0, 1, 1, 1])&(result1[max_index][max_index1][0]==[0, 0, 0, 0, 1, 1, 1])&(result2[max_index][max_index1][max_index2][0]==[1, 1, 1, 0, 1, 1, 1]):
                same+=1
            else:
                different+=1

最大利润为:
2167532.7652249634
第一阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 1483643.0738264269, [0.095, 0.095, 0.095], 3291.0]
第二阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 488266.93559627706, [0.095, 0.095, 0.095], 1083.0]
第三阶段的决策为:
[[1, 1, 1, 0, 1, 1, 1], 195622.75580225972, [0.0, 0.0, 0.0], 93.0]
最大利润为:
2165518.991618227
第一阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 1482360.235635988, [0.095, 0.095, 0.095], 3291.0]
第二阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 487844.75354780356, [0.095, 0.095, 0.095], 1083.0]
第三阶段的决策为:
[[1, 1, 1, 0, 1, 1, 1], 195314.00243443582, [0.0, 0.0, 0.0], 93.0]
最大利润为:
2163505.218011491
第一阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 1481077.3974455488, [0.095, 0.095, 0.095], 3291.0]
第二阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 487422.57149933005, [0.095, 0.095, 0.095], 1083.0]
第三阶段的决策为:
[[1, 1, 1, 0, 1, 1, 1], 195005.24906661187, [0.0, 0.0, 0.0], 93.0]
最大利润为:
2161491.444404755
第一阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 1479794.55925511, [0.095, 0.095, 0.095], 3291.0]
第二阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 487000.3894508567, [0.095, 0.095, 0.095], 

In [6]:
print(f"在0.95-1.05波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.95-1.05波动时,决策差异率为:{different/125}")

在0.95-1.05波动时,生产决策方案和原方案一致的次数为:125
在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:0
在0.95-1.05波动时,决策差异率为:0.0


In [7]:
P_part_sample=0.1
P_semi_product_sample=0.1
P_product_sample=0.1
same=0
different=0
for P_part in np.linspace(0.9*P_part_sample,1.1*P_part_sample,5):
    for P_semi_product in np.linspace(0.9*P_semi_product_sample,1.1*P_semi_product_sample,5):
        for P_product in np.linspace(0.9*P_product_sample,1.1*P_product_sample,5):
            result=[]
            for i in decision:
                P_init=[P_part for i in range(3)]
                n0=10000
                profit,P_upgrade,n_upgrade=calculate_profit(i,P_init,n0,P_semi_product,P_product)
                result.append([i,profit,P_upgrade,n_upgrade])
            non_zero_index=[]
            for i in range(2**7):
                if (result[i][3]!=0)&(result[i][1]>0):
                    non_zero_index.append(i)

            result1=np.empty((2**7,2**7), dtype=object)
            for i in non_zero_index:
                for j in range(2**7):
                    profit,P_upgrade,n_upgrade=calculate_profit(decision[j],result[i][2],result[i][3],P_semi_product,P_product)
                    result1[i,j]=[decision[j],profit,P_upgrade,n_upgrade]
            non_zero_index1=[]
            for i in non_zero_index:
                for j in range(2**7):
                    if (result1[i][j][3]!=0)&(result1[i][j][1]>0):
                        non_zero_index1.append([i,j])

            result2=np.empty((2**7,2**7,2**7), dtype=object)
            for i in non_zero_index1:
                for j in range(2**7):
                    profit,P_upgrade,n_upgrade=calculate_profit(decision[j],result1[i[0]][i[1]][2],result1[i[0]][i[1]][3],P_semi_product,P_product)
                    result2[i[0],i[1],j]=[decision[j],profit,P_upgrade,n_upgrade]

            result2_value=np.zeros((2**7,2**7,2**7))
            for i in non_zero_index1:
                for j in range(2**7):
                    result2_value[i[0]][i[0]][j]=result2[i[0]][i[0]][j][1]
            result2_value_max=np.max(result2_value,axis=2)

            result1_value=np.zeros((2**7,2**7))
            for i in non_zero_index:
                for j in range(2**7):
                    result1_value[i,j]=result1[i,j][1]
                    result1_value[i,j]=result1[i,j][1]+result2_value_max[i,j]
            result1_value_max=np.max(result1_value,axis=1)
            
            result_value=np.zeros(2**7)
            for i in range(2**7):
                result_value[i]=result[i][1]
                result_value[i]=result_value[i]+result1_value_max[i]
            max_index = np.argmax(result_value)
            max_value = result_value[max_index]
            print("最大利润为:")
            print(max_value)
            print("第一阶段的决策为:")
            print(result[max_index])
            max_index1= np.argmax(result1_value[max_index])
            print("第二阶段的决策为:")
            print(result1[max_index][max_index1])
            max_index2= np.argmax(result2_value[max_index][max_index1])
            print("第三阶段的决策为:")
            print(result2[max_index][max_index1][max_index2])
            if (result[max_index][0]==[0, 0, 0, 0, 1, 1, 1])&(result1[max_index][max_index1][0]==[0, 0, 0, 0, 1, 1, 1])&(result2[max_index][max_index1][max_index2][0]==[1, 1, 1, 0, 1, 1, 1]):
                same+=1
            else:
                different+=1

最大利润为:
2173112.9250428793
第一阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 1515434.6492288467, [0.09000000000000001, 0.09000000000000001, 0.09000000000000001], 3142.0]
第二阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 476149.56678770366, [0.09000000000000001, 0.09000000000000001, 0.09000000000000001], 987.0]
第三阶段的决策为:
[[1, 1, 1, 0, 1, 1, 1], 181528.70902632904, [0.0, 0.0, 0.0], 80.0]
最大利润为:
2168935.338480319
第一阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 1512693.607585831, [0.09000000000000001, 0.09000000000000001, 0.09000000000000001], 3142.0]
第二阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 475288.3315034681, [0.09000000000000001, 0.09000000000000001, 0.09000000000000001], 987.0]
第三阶段的决策为:
[[1, 1, 1, 0, 1, 1, 1], 180953.3993910195, [0.0, 0.0, 0.0], 80.0]
最大利润为:
2164757.7519177576
第一阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 1509952.565942815, [0.09000000000000001, 0.09000000000000001, 0.09000000000000001], 3142.0]
第二阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 474427.09621923254, [0.09000000000000001, 0.09000000000000001, 0.09000000000000001], 987.0]
第三阶段的决策为:
[[1,

In [8]:
print(f"在0.9-1.1波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.9-1.1波动时,决策差异率为:{different/125}")

在0.9-1.1波动时,生产决策方案和原方案一致的次数为:125
在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:0
在0.9-1.1波动时,决策差异率为:0.0


In [9]:
P_part_sample=0.1
P_semi_product_sample=0.1
P_product_sample=0.1
same=0
different=0
for P_part in np.linspace(0.8*P_part_sample,1.2*P_part_sample,5):
    for P_semi_product in np.linspace(0.8*P_semi_product_sample,1.2*P_semi_product_sample,5):
        for P_product in np.linspace(0.8*P_product_sample,1.2*P_product_sample,5):
            result=[]
            for i in decision:
                P_init=[P_part for i in range(3)]
                n0=10000
                profit,P_upgrade,n_upgrade=calculate_profit(i,P_init,n0,P_semi_product,P_product)
                result.append([i,profit,P_upgrade,n_upgrade])
            non_zero_index=[]
            for i in range(2**7):
                if (result[i][3]!=0)&(result[i][1]>0):
                    non_zero_index.append(i)

            result1=np.empty((2**7,2**7), dtype=object)
            for i in non_zero_index:
                for j in range(2**7):
                    profit,P_upgrade,n_upgrade=calculate_profit(decision[j],result[i][2],result[i][3],P_semi_product,P_product)
                    result1[i,j]=[decision[j],profit,P_upgrade,n_upgrade]
            non_zero_index1=[]
            for i in non_zero_index:
                for j in range(2**7):
                    if (result1[i][j][3]!=0)&(result1[i][j][1]>0):
                        non_zero_index1.append([i,j])

            result2=np.empty((2**7,2**7,2**7), dtype=object)
            for i in non_zero_index1:
                for j in range(2**7):
                    profit,P_upgrade,n_upgrade=calculate_profit(decision[j],result1[i[0]][i[1]][2],result1[i[0]][i[1]][3],P_semi_product,P_product)
                    result2[i[0],i[1],j]=[decision[j],profit,P_upgrade,n_upgrade]

            result2_value=np.zeros((2**7,2**7,2**7))
            for i in non_zero_index1:
                for j in range(2**7):
                    result2_value[i[0]][i[0]][j]=result2[i[0]][i[0]][j][1]
            result2_value_max=np.max(result2_value,axis=2)

            result1_value=np.zeros((2**7,2**7))
            for i in non_zero_index:
                for j in range(2**7):
                    result1_value[i,j]=result1[i,j][1]
                    result1_value[i,j]=result1[i,j][1]+result2_value_max[i,j]
            result1_value_max=np.max(result1_value,axis=1)
            
            result_value=np.zeros(2**7)
            for i in range(2**7):
                result_value[i]=result[i][1]
                result_value[i]=result_value[i]+result1_value_max[i]
            max_index = np.argmax(result_value)
            max_value = result_value[max_index]
            print("最大利润为:")
            print(max_value)
            print("第一阶段的决策为:")
            print(result[max_index])
            max_index1= np.argmax(result1_value[max_index])
            print("第二阶段的决策为:")
            print(result1[max_index][max_index1])
            max_index2= np.argmax(result2_value[max_index][max_index1])
            print("第三阶段的决策为:")
            print(result2[max_index][max_index1][max_index2])
            if (result[max_index][0]==[0, 0, 0, 0, 1, 1, 1])&(result1[max_index][max_index1][0]==[0, 0, 0, 0, 1, 1, 1])&(result2[max_index][max_index1][max_index2][0]==[1, 1, 1, 0, 1, 1, 1]):
                same+=1
            else:
                different+=1

最大利润为:
2189205.3247720096
第一阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 1586079.4974922356, [0.08000000000000002, 0.08000000000000002, 0.08000000000000002], 2836.0]
第二阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 449812.14548879804, [0.08000000000000002, 0.08000000000000002, 0.08000000000000002], 804.0]
第三阶段的决策为:
[[1, 1, 1, 0, 1, 1, 1], 153313.68179097597, [0.0, 0.0, 0.0], 59.0]
最大利润为:
2180203.237095987
第一阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 1579829.1689021026, [0.08000000000000002, 0.08000000000000002, 0.08000000000000002], 2836.0]
第二阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 448039.5523006363, [0.08000000000000002, 0.08000000000000002, 0.08000000000000002], 804.0]
第三阶段的决策为:
[[1, 1, 1, 0, 1, 1, 1], 152334.515893248, [0.0, 0.0, 0.0], 59.0]
最大利润为:
2171201.1494199643
第一阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 1573578.8403119696, [0.08000000000000002, 0.08000000000000002, 0.08000000000000002], 2836.0]
第二阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 446266.9591124746, [0.08000000000000002, 0.08000000000000002, 0.08000000000000002], 804.0]
第三阶段的决策为:
[[1,

In [10]:
print(f"在0.8-1.2波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.8-1.2波动时,决策差异率为:{different/125}")

在0.8-1.2波动时,生产决策方案和原方案一致的次数为:125
在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:0
在0.8-1.2波动时,决策差异率为:0.0


7
2163747.091137528
第一阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 1454019.1908159298, [0.1, 0.1, 0.1], 3439.0]


7
第二阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 500037.19972159824, [0.1, 0.1, 0.1], 1182.0]


119
第三阶段的决策为:
[[1, 1, 1, 0, 1, 1, 1], 209690.70059999998, [0.0, 0.0, 0.0], 106.0]
